In [1]:

from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

In [10]:
from data_loading import load_data_using_sqlalchemy
from user_engagement import (
    aggregate_user_metrics,
    normalize_metrics,
    cluster_users
)
from experience_analytics import (
    aggregate_experience_metrics
)
from satisfaction_analysis import (
    calculate_engagement_score,
    calculate_experience_score,
    calculate_satisfaction_score,
    top_satisfied_customers,
    train_regression_model,
    cluster_satisfaction_scores,
    aggregate_cluster_scores,
    export_to_mysql
)


In [4]:
# Load environment variables from .env file
load_dotenv()

# Fetch database connection parameters from environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [5]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
df =  load_data_using_sqlalchemy(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")


Successfully loaded the data


In [6]:
# Step 2: Engagement and experience data
engagement_columns = ['Session Duration (s)', 'Total Traffic (Bytes)']
experience_columns = ['Avg TCP Retrans (Bytes)', 'Avg RTT (ms)', 'Avg Throughput (kbps)']


In [12]:
# Use Task 2 and Task 3 outputs
user_engagement = aggregate_user_metrics(df)
user_engagement_normalized = normalize_metrics(user_engagement, engagement_columns)
user_engagement_clustered = cluster_users(user_engagement_normalized, engagement_columns, n_clusters=3)

# Step 2: Aggregate experience metrics
experience_metrics = aggregate_experience_metrics(df)
print(experience_metrics.head())
experience_clustered, _ = perform_experience_clustering(experience_metrics, experience_columns)


   MSISDN/Number  Avg TCP Retrans (Bytes)  Avg RTT (ms)  \
0   3.360100e+10             1.685339e+07     46.000000   
1   3.360100e+10             1.685339e+07     30.000000   
2   3.360100e+10             1.685339e+07    119.182869   
3   3.360101e+10             1.066000e+03     69.000000   
4   3.360101e+10             9.349630e+06     57.000000   

   Avg Throughput (kbps)                    Handset Type  
0                   37.0  Huawei P20 Lite Huawei Nova 3E  
1                   48.0          Apple iPhone 7 (A1778)  
2                   48.0                       undefined  
3                  204.0         Apple iPhone 5S (A1457)  
4                20197.5         Apple iPhone Se (A1723)  


In [13]:
# Merge engagement and experience data
df_combined = pd.merge(user_engagement_clustered, experience_clustered, on='MSISDN/Number')

In [14]:
# Step 3: Calculate engagement and experience scores
df_combined = calculate_engagement_score(df_combined, engagement_columns, reference_cluster=2)  # Assuming cluster 2 is less engaged
df_combined = calculate_experience_score(df_combined, experience_columns, reference_cluster=1)  # Assuming cluster 1 is worst experience

In [15]:
# Step 4: Satisfaction score
df_combined = calculate_satisfaction_score(df_combined)

In [16]:
# Step 5: Top 10 satisfied customers
top_customers = top_satisfied_customers(df_combined, n=10)
print("\nTop 10 Satisfied Customers:\n", top_customers)


Top 10 Satisfied Customers:
        MSISDN/Number  Satisfaction Score
85785   3.369858e+10        2.138491e+09
32273   3.365871e+10        2.137778e+09
58026   3.366491e+10        2.127964e+09
31942   3.365863e+10        2.121070e+09
62049   3.366613e+10        2.099342e+09
70079   3.366877e+10        2.077043e+09
43629   3.366131e+10        2.059270e+09
48341   3.366240e+10        2.052624e+09
64113   3.366682e+10        1.977784e+09
98211   3.376264e+10        1.886395e+09


In [17]:
# Step 6: Train regression model
features = ['Engagement Score', 'Experience Score']
target = 'Satisfaction Score'
model = train_regression_model(df_combined, features, target)


In [18]:
# Step 7: Cluster satisfaction scores
df_combined = cluster_satisfaction_scores(df_combined, ['Engagement Score', 'Experience Score'], n_clusters=2)


In [19]:
# Step 8: Aggregate scores per cluster
cluster_scores = aggregate_cluster_scores(df_combined)
print("\nCluster Satisfaction Scores:\n", cluster_scores)


Cluster Satisfaction Scores:
    Satisfaction Cluster  Satisfaction Score  Experience Score
0                     0        4.981266e+06      9.962533e+06
1                     1        1.105499e+09      2.210999e+09


In [21]:
# Step 9: Export to MySQL
connection_string = f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
export_to_mysql(df_combined, 'satisfaction_scores', connection_string)